# Calculate reproduction rate (R0) for infectious diseases within a micro-environment
This model calculates the reproduction rate (R0) of infectious diseases (Covid19) within a micro-environment based upon dispersion of an aerosol within an enclosed space. It assumes the prime method of transmission is an aerosol and that the aerosol is well mixed across the space, i.e. there are no local concentrations which may impact on an individual's probability of being infected.

The modelling follows the approach set out in the paper:
Buonanno, G., Stabile, L., & Morawska, L. (2020). Estimation of airborne viral emission: Quanta emission rate of SARS-CoV-2 for infection risk assessment [Preprint]. Infectious Diseases (except HIV/AIDS). https://doi.org/10.1101/2020.04.12.20062828

In [25]:
import math
import plotly.graph_objects as go
import simpy

# Import local libraries
from Tools.Check import Check
from Microenvironment import Microenvironment

In [26]:
# Create a simpy environment
env = simpy.Environment()

In [27]:
# Plot concentration in micro-environment over a period

# Set base statistics
me_volume = 75 # m^3
me_air_exchange_rate = 2.2  # h^-1: natural ventilation (0.2) mechanical ventilation (2.2)
me_quanta_emission_rate = 142 # quanta h^-1: Resting(98.1), Standing (147), Light Exercise (317)
me_n0 = 0.0

me_infected = 1

periods = 180
time_interval = 1/60 # minutes

microenvironment = Microenvironment(env, time_interval, me_volume, me_air_exchange_rate)
env.process(microenvironment.simulate())
env.process(microenvironment.infected_person(me_quanta_emission_rate, 10))
env.run(until=periods)

time_axis = list(range(periods))
results = microenvironment.get_results()

fig = go.Figure()
fig.add_trace(go.Scatter(x=time_axis, y=results , name='quanta concentration', line=dict(color='firebrick', width=1)))
fig.update_layout(title='Quanta concentration',
                   xaxis_title='Time (minutes)',
                   yaxis_title='Quanta concentration (quanta per m^-3)',
                   width=1280,
                   height=720)
fig.show()

In [28]:
# Calculate the infection risk as a function of exposure time using the Wells-Riley equation
def infection_risk(exposure_time, inhalation_rate):
    """ Calculate the infection risk based upon exposure time and inhalation rate

    Keyword arguments:
    exposure_time           Amount of time that the susceptible person was exposed
    inhalation_rate         Inhalation rate of the exposed subject (affected by subjects activity levels)

    Return value:
    infection_risk          Risk that a susceptible person will be infected as a probability
    """

    # Assume parameters are valid (all parameters greater than zero)

    # Integrate quanta_concentration over time from 0 to T [exposure_time]
    